<h1 style="text-align: center;">Homework 3 - Model-free control</h1>
In the last homework we dealt with model-free prediction. We evaluated a given policy. In this homework we deal with Model-free control. The goal is to optimize the policy to maximize return.
<h1 style="text-align: center;">Imports</h1>

In [ ]:
import collections
import gym
import random
import numpy as np
import matplotlib.pyplot as plt
import operator
import pandas as pd
import statistics

<h1 style="text-align: center;">Helper functions</h1>

Helper functions such as epsilon greedy are required for Sarsa and Q-learning. We also need a helper function to evaluate statistics.

In [ ]:
def choose_action_epsilon_greedy(state, epsilon, q_map, actions):
    if random.uniform(0, 1) <= epsilon:
        return random.randrange(0, actions)
    else:
        return np.argmax(q_map[state])

def choose_action_greedy(state, q_map, actions):
    return choose_action_epsilon_greedy(state, 0, q_map, actions)

def print_greedy_policy(q_map, actions_utf_map, actions, grid_width = 4):
    result = ''
    for state in range(0, len(q_map)):
        action = choose_action_greedy(state, q_map, actions)
        result += '\n' if state % grid_width == 0 else ' '
        result += f'{actions_utf_map[action]} '
    print(result)
    
def plot_episode_statistics(data_map, alpha, episodes, algorithm, exercise, ylabel):
    plt.figure(figsize=(20, 8))
    for key, value in data_map.items():
        df = pd.DataFrame({'x': np.arange(1, episodes+1), 'y': data_map[key]})
        rolling_means = df.rolling(rolling_mean_size).mean()
        plt.plot('x', 'y', f'{marker}{line}', data=rolling_means, label=f'alpha={alpha}, epsilon={key}')
    plt.legend()
    plt.title(f'{exercise} - {algorithm["name"]}')
    plt.xlabel('# episodes')
    plt.ylabel(ylabel)
    plt.show()
    
def plot_episode_statistics_sampled(data_map, alpha, episodes, algorithm, exercise, ylabel):
    plt.figure(figsize=(20, 8))
    for key, value in data_map.items():
        df = pd.DataFrame({'x': np.arange(1, episodes+1), 'y': data_map[key]})
        df_sample = df.sample(sample_size).sort_index()
        plt.plot('x', 'y', f'{marker}{line}', data=df_sample, label=f'alpha={alpha}, epsilon={key}')
    plt.legend()
    plt.title(f'{exercise} - {algorithm["name"]}')
    plt.xlabel('# episodes')
    plt.ylabel(ylabel)
    plt.show()

<h1 style="text-align: center;">Sarsa(0)</h1>

SARSA is the most simple TD learning algorithm, named after <b>S</b>tate <b>a</b>ction <b>r</b>eward <b>s</b>tate <b>a</b>ction.

Whenever we observe a reward, we update the previous q-value based on that reward and the discounted estimated value of the next action (that will be chosen under a policy based on q).

In the current state s take an a, selected based on q-function. Therefore  we need a q_map with state and action, q(s, a). We update q(s, a) based on observed reward and estimated q-value for the next action.

In [ ]:
def sarsa_zero(exercise, number_of_episodes, epsilon, gamma, alpha):
    env = gym.make(exercise)
    action_count = env.nA
    print(f'# states: {env.nS}')
    print(f'# actions: {action_count}')
    q_map = np.zeros((env.nS, env.nA))
    successful_episodes = 0
    step_counts = []
    rewards = []
    for episode in range(1, number_of_episodes + 1):
        state = env.reset()
        episode_finished = False
        a = choose_action_epsilon_greedy(state, epsilon, q_map, action_count)
        step_count = 0
        cumulated_reward = 0
        while not episode_finished:
            new_state, reward, done, _ = env.step(a)
            cumulated_reward += reward
            if done and reward >= 1:
                successful_episodes += 1
            episode_finished = done
            a_next = choose_action_epsilon_greedy(new_state, epsilon, q_map, action_count)
            td_error = reward + gamma * q_map[new_state, a_next] - q_map[state, a]
            q_map[state, a] = q_map[state, a] + alpha * td_error

            state = new_state
            a = a_next
            step_count += 1
        step_counts.append(step_count)
        rewards.append(cumulated_reward)
    print(f'Successful episodes: {successful_episodes}/{number_of_episodes} ({successful_episodes/number_of_episodes})')
    print(f'Mean steps per episode: {statistics.mean(step_counts)}')
    print(f'Median steps per episode: {statistics.median(step_counts)}')
    return q_map, step_counts, rewards

<h1 style="text-align: center;">Sarsa(λ)</h1>

Sarsa lambda is very similar to sarsa. There is a difference in the use of state-action eligibility traces. For example accumulating traces, dutch traces and replacing traces.
The updates to the q-function are based on the TD-error and the eligibility trace values.

In [ ]:
def sarsa_lambda(exercise, number_of_episodes, epsilon, gamma, alpha, lambda_value):
    env = gym.make(exercise)
    action_count = env.nA
    print(f'# states: {env.nS}')
    print(f'# actions: {action_count}')
    q_map = np.zeros((env.nS, env.nA))
    successful_episodes = 0
    step_counts = []
    rewards = []
    for episode in range(1, number_of_episodes + 1):
        eligibility_traces = np.zeros((env.nS, env.nA))
        state = env.reset()
        episode_finished = False
        a = choose_action_epsilon_greedy(state, epsilon, q_map, action_count)
        step_count = 0
        cumulated_reward = 0
        while not episode_finished:
            new_state, reward, done, _ = env.step(a)
            cumulated_reward += reward
            if done and reward >= 1:
                successful_episodes += 1
            episode_finished = done

            a_next = choose_action_epsilon_greedy(new_state, epsilon, q_map, action_count)
            delta = reward + gamma * q_map[new_state, a_next] - q_map[state, a]

            eligibility_traces[state, a] += 1 # accumulating traces
            # or eligibility_traces[state, a] = (1-alpha) * eligibility_traces[state, a] +1 # dutch traces
            # or eligibility_traces[state, a] = 1 # replacing traces
            for es in range(0, env.nS):
                for ea in range (0, env.nA):
                    q_map[state, a] = q_map[state, a] + alpha * delta * eligibility_traces[es, ea]
                    eligibility_traces[es, ea] = gamma * lambda_value * eligibility_traces[es, ea]

            state = new_state
            a = a_next
            step_count += 1        
        step_counts.append(step_count)
        rewards.append(cumulated_reward)
    print(f'Successful episodes: {successful_episodes}/{number_of_episodes} ({successful_episodes/number_of_episodes})')
    print(f'Mean steps per episode: {statistics.mean(step_counts)}')
    print(f'Median steps per episode: {statistics.median(step_counts)}')
    return q_map, step_counts, rewards

<h1 style="text-align: center;">Q-Learning</h1>

Q-learning is similar to Sarsa. The key difference is that Sarsa is On-Policy and Q-learning is Off-Policy learning. The q-value of the action that would be selected by the target policy.

In [ ]:
def q_learning(exercise, number_of_episodes, epsilon, gamma, alpha):
    env = gym.make(exercise)
    action_count = env.nA
    print(f'# states: {env.nS}')
    print(f'# actions: {action_count}')
    q_map = np.zeros((env.nS, env.nA))
    successful_episodes = 0
    step_counts = []
    rewards = []
    for episode in range(1, number_of_episodes + 1):
        state = env.reset()
        episode_finished = False;
        step_count = 0
        cumulated_reward = 0
        while not episode_finished:
            a = choose_action_epsilon_greedy(state, epsilon, q_map, action_count)

            new_state, reward, done, _ = env.step(a)
            cumulated_reward += reward
            if done and reward >= 1:
                successful_episodes += 1
            episode_finished = done

            a_next = choose_action_greedy(new_state, q_map, action_count)
            td_error = reward + gamma * q_map[new_state, a_next] - q_map[state, a]
            q_map[state, a] = q_map[state, a] + alpha * td_error

            state = new_state
            step_count += 1
        step_counts.append(step_count)
        rewards.append(cumulated_reward)
    print(f'Successful episodes: {successful_episodes}/{number_of_episodes} ({successful_episodes/number_of_episodes})')
    print(f'Mean steps per episode: {statistics.mean(step_counts)}')
    print(f'Median steps per episode: {statistics.median(step_counts)}')
    return q_map, step_counts, rewards

<h1 style="text-align: center;">Example problems</h1>

The evolution of The Frozen Lake example and The Taxi example is shown below. We'll use the following parameters for both examples to better compare them.

In [ ]:
# Hyperparameter
episodes = 10000
gamma = 1
alphas = [0.2, 0.1, 0.05]
epsilons= [0.1, 0.2, 0.3, 0.5]
algorithms = [
    {
        "name": "Sarsa(0)",
        "q_map": lambda alpha, epsilon: sarsa_zero(exercise, episodes, epsilon, gamma, alpha),
    },
    {
        "name": "Sarsa(λ = 0)",
        "q_map": lambda  alpha, epsilon: sarsa_lambda(exercise, episodes, epsilon, gamma, alpha, 0),
    },
    {
        "name": "Sarsa(λ = 0.5)",
        "q_map": lambda  alpha, epsilon: sarsa_lambda(exercise, episodes, epsilon, gamma, alpha, 0.5)
    },
    {
        "name": "Sarsa(λ = 1)",
        "q_map": lambda  alpha, epsilon: sarsa_lambda(exercise, episodes, epsilon, gamma, alpha, 1)
    },
    {
        "name": "Q-Learning",
        "q_map": lambda alpha, epsilon: q_learning(exercise, episodes, epsilon, gamma, alpha)
    }
]

# Matplotlib
sample_size = 200
rolling_mean_size = 250
marker = ','
line = '-'

<h2 style="text-align: center;">FrozenLake-v0</h2>

The agent controls the movement of a character in a grid world. Some tiles of the grid are walkable, and others lead to the agent falling into the water. Additionally, the movement direction of the agent is uncertain and only partially depends on the chosen direction. The agent is rewarded for finding a walkable path to a goal tile.

*Winter is here. You and your friends were tossing around a frisbee at the park when you made a wild throw that left the frisbee out in the middle of the lake. The water is mostly frozen, but there are a few holes where the ice has melted. If you step into one of those holes, you'll fall into the freezing water. At this time, there's an international frisbee shortage, so it's absolutely imperative that you navigate across the lake and retrieve the disc. However, the ice is slippery, so you won't always move in the direction you intend.*

*The surface is described using a grid like the following:*


```
SFFF       (S: starting point, safe)
FHFH       (F: frozen surface, safe)
FFFH       (H: hole, fall to your doom)
HFFG       (G: goal, where the frisbee is located)
```

The episode ends when you reach the goal or fall in a hole. You receive a reward of 1 if you reach the goal, and zero otherwise.

Source: https://gym.openai.com/envs/FrozenLake-v0/

In [ ]:
exercise = 'FrozenLake-v0'

LEFT = 0
DOWN = 1
RIGHT = 2
UP = 3

actions_utf_map = {
    LEFT: '←',
    DOWN:  '↓',
    RIGHT:  '→',
    UP: '↑',
}

for algorithm in algorithms:
    for alpha in alphas:
        step_count_map = {}
        rewards_map = {}
        for epsilon in epsilons:
            print(algorithm["name"] + " - α: "+ str(alpha) + " - ɛ: "+ str(epsilon))
            q_map, step_counts, rewards = algorithm["q_map"](alpha = alpha, epsilon = epsilon)
            step_count_map[epsilon] = step_counts
            rewards_map[epsilon] = rewards
            print_greedy_policy(q_map, actions_utf_map, 4)
            print("\n\n")
    
        plot_episode_statistics(step_count_map, alpha, episodes, algorithm, exercise, 'steps per episode')
        plot_episode_statistics(rewards_map, alpha, episodes, algorithm, exercise, 'reward per episode')

<h1 style="text-align: center;">Taxi-v3</h1>

This task was introduced in [Dietterich2000] to illustrate some issues in hierarchical reinforcement learning. There are 4 locations (labeled by different letters) and your job is to pick up the passenger at one location and drop him off in another. You receive +20 points for a successful dropoff, and lose 1 point for every timestep it takes. There is also a 10 point penalty for illegal pick-up and drop-off actions.

Source: https://gym.openai.com/envs/Taxi-v3/

<img src="https://storage.googleapis.com/lds-media/images/Reinforcement_Learning_Taxi_Env.width-1200.png" style="width: 400px;"/>

In [ ]:
# see https://github.com/openai/gym/blob/master/gym/envs/toy_text/taxi.py (l 134 ff.)
x_factor = 100
y_factor = 20

# Calculates the positional part of the state
def compute_positional_state_part(x, y):
    return x * x_factor + y * y_factor


def get_states_for_ride(passenger_location, passenger_destination):
    states = []
    for taxi_x in range(0, 5):
        for taxi_y in range(0, 5):
            state = compute_positional_state_part(taxi_x, taxi_y) + passenger_location * 4 + passenger_destination
            states.append(state)
    return states


# prints the action the the highest value for each state that corresponds to the given passenger location and destination
def print_taxi_policy(q_map, actions_utf_map, passenger_location, passenger_destination):
    relevant_states = get_states_for_ride(passenger_location, passenger_destination)
    relevant_q_values = operator.itemgetter(*relevant_states)(q_map)
    print_greedy_policy(relevant_q_values, actions_utf_map, 6, 5)

In [ ]:
exercise = 'Taxi-v3'

# Actions
SOUTH = 0
NORTH = 1
EAST = 2
WEST = 3
PICK_UP = 4
DROP_OFF = 5

actions_utf_map = {
    SOUTH:  '↓',
    NORTH: '↑',
    EAST: '→',
    WEST:  '←',
    PICK_UP: 'P',
    DROP_OFF: 'D',
}

## Passenger locations
PASSENGER_AT_R = 0
PASSENGER_AT_G = 1
PASSENGER_AT_Y = 2
PASSENGER_AT_B = 3
PASSENGER_IN_TAXI = 4

## Destinations
DEST_R = 0
DEST_G = 1
DEST_Y = 2
DEST_B = 3

for algorithm in algorithms:
    for alpha in alphas:
        step_count_map = {}
        rewards_map = {}
        for epsilon in epsilons:
            print(algorithm["name"] + " - α: "+ str(alpha) + " - ɛ: "+ str(epsilon))
            q_map, step_counts, rewards = algorithm["q_map"](alpha = alpha, epsilon = epsilon)
            step_count_map[epsilon] = step_counts
            rewards_map[epsilon] = rewards

            print('Passenger starts at B with target R')
            print_taxi_policy(q_map, actions_utf_map, PASSENGER_AT_B, DEST_R)
            print_taxi_policy(q_map, actions_utf_map, PASSENGER_IN_TAXI, DEST_R)


            print('Passenger starts at B with target Y')
            print_taxi_policy(q_map, actions_utf_map, PASSENGER_AT_B, DEST_Y)
            print_taxi_policy(q_map, actions_utf_map, PASSENGER_IN_TAXI, DEST_Y)
            print("\n\n")
    
        plot_episode_statistics(step_count_map, alpha, episodes, algorithm, exercise, 'steps per episode')
        plot_episode_statistics(rewards_map, alpha, episodes, algorithm, exercise, 'reward per episode')